<a href="https://colab.research.google.com/github/AllieUbisse/end-to-end-ml/blob/master/end_to_end_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>